# Amplitude Amplification Algorithm

Now we have all ingredientes to use amplitude amplification for computing the expected value of a function $f(x)$ over a domain that follows a distribution probability $p(x)$: $E_{x\sim p}(f)$:

1. Loading distribution probability $p(x)$ using the operator $\mathcal{P}$ into the quantum state.
2. Loading the desired function $f(x)$ using the operator $\mathcal{R}$ into the quantum state.
3. The Groover operator mandatory for Amplitude Amplification: $\mathcal{Q}$

In this notebook we try to use the **Groover** operator for calculating $E_{x\sim p}(f)$ and analyze the problems that arise when using this method

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [ ]:
%load_ext qat.core.magic

In [ ]:
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()    

In [ ]:
from QuantumMultiplexors_Module import expectation_loading_data
from PhaseAmplification_Module import load_q_gate, load_qn_gate

## 1. Data Discretization

First we create the discretized probability $p(x)$ and function $f(x)$ needed for calculating expected value $E_{x\sim p}(f)$

In [ ]:
from AuxiliarFunctions import  get_histogram, postprocess_results, test_bins, run_job
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 6
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

## 2. $\theta$ estimation problem

After the loading steps our quantum state is: 

$$|\Psi\rangle=\sin{\theta}|\Psi_{1}\rangle+\cos{\theta}|\Psi_{0}\rangle$$

The amplitude of the $|\Psi_{1}\rangle$ state is realted with $E_{x\sim p}(f)$ by:

$$\sin \theta = \sqrt{\sum_{x=0}^{2^-1} p(x)f(x)}$$

The main idea in the amplitude amplification is use a similar procedure like in Groover algorithm in order to increase the probability of the $|\Psi_{1}\rangle$ state. This can be in a straightoforward way by applying operator $\hat{Q}$ $k$ times:

$$\hat{Q}^{K}|\Psi\rangle = \sin{((2*K+1)\theta)}|\Psi_{1}\rangle+\cos{((2*K+1)\theta})|\Psi_{0}\rangle$$

The idea is apply K in such a way that the probability of $|\Psi_{1}\rangle$ is maximized. For this we can equal:


$$P_{|\Psi_{1}\rangle} = \sin^{2}((2*K+1)\theta) \approx 1$$ 

So:

$$(2K+1)\theta = \frac{\pi}{2}$$


We have to unknows variables: $\theta$ and $K$. If we know $\theta$ we can know how many applications of $\hat{Q}$ we need for maximazing the state we want.

Other approximation is following one: 

we can prepare the system $n$ times (each time with the same $K$), measure the final state and get the probabilities for measuring  $|\Psi_{1}\rangle$ y $|\Psi_{1}:

* $P_{|\Psi_{1}\rangle}$: Probability of get state $|\Psi_{1}\rangle$ (we desire a high probability here)
* $P_{|\Psi_{0}\rangle}$: Probability of get state $|\Psi_{0}\rangle$ (we desire a low probability here)

We know that this proabilities are related withe the corresponding amplitudes so: 

$$\sin^{2}((2*K+1)\theta) = P_{|\Psi_{1}\rangle}$$
$$\sin((2*K+1)\theta) = \sqrt{P_{|\Psi_{1}\rangle}}$$
$$(2*K+1)\theta = \arcsin{\sqrt{P_{|\Psi_{1}\rangle}}}$$

So in this moment, theoretically, we have solved the problem because we can calculate $\theta$ and the desired $E_{x\sim p}(f)$. But we have a practical important problem: **arcsin** is a multivaluated function and usually software packages give a solution in the $[\frac{-\pi}{2}, \frac{\pi}{2}]$. If the rotation resulting of apply $\hat{Q}^{K}$ is bigger than $\frac{\pi}{2}$ then we cannot compute properly $(2*K+1)\theta$.

This can be seen in the following cells

In [ ]:
from qat.lang.AQASM import Program

In [ ]:
#Number of times operator Grover-like Q will be applied
K=2

#Gate for loading data staff
PR_gate = expectation_loading_data(p_X, f_X)
#Groover Operator Gate
Q_Gate = load_q_gate(PR_gate)

#Loading Data in Quantum Circuit
Qprog = Program()
qbits = Qprog.qalloc(PR_gate.arity)
Qprog.apply(PR_gate, qbits)
#Applications of Groover Operator K times
step_q_gate = load_qn_gate(Q_Gate, K)
Qprog.apply(step_q_gate, qbits)

circuit = Qprog.to_circ(submatrices_only=True)
job = circuit.to_job(qubits=[n_qbits])

In [ ]:
%qatdisplay circuit --depth 1

In [ ]:
#Getting the final state
result = run_job(lineal_qpu.submit(job))
Phi = postprocess_results(result)

In [ ]:
Phi

Now we calculate:  
$$\theta = \frac{\arcsin{\sqrt{P_{|\Psi_{1}\rangle}}}}{(2*K+1)}$$

In [ ]:
#This is the angle asociated to the Phi state
theta_K = np.arcsin((Phi['Probability'].iloc[1])**0.5)
print('theta_K: {}'.format(theta_K))
thetaFromK = theta_K/(2*K+1)
print('thetaFromK: {}'.format(thetaFromK))

Additionally in this case we know the true value of $\theta$:

$$ \theta_{th} = \arcsin{\sqrt{\sum_{x=0}^{2^-1} p(x)f(x)}}$$

This is done in following cell

In [ ]:
#True value of Theta calculated using the input arrays
theoric_theta = np.arcsin(np.sqrt(sum(p_X*f_X)))
print('theoric_theta: {}'.format(theoric_theta))

print('thetaFromK == theoric_theta? {}'.format(np.isclose(thetaFromK, theoric_theta)))

As can be seen in befor cell the calculated $\theta$ is different from $\theta_{th}$. But we can compute the probability associated with this $\theta_{th}$ using:

$$\sin^2((2*K+1)\theta_{th}$$

In [ ]:
print('|Psi_1> from  theoric_theta: {}'.format(np.sin((2*K+1)*theoric_theta)**2))
print('|Psi_1> from  Q^KPhi Probability: {}'.format(Phi['Probability'].iloc[1]))
print('Probability from Q^Phi and from theoric_theta are the same? {}'.format(
    np.isclose(np.sin((2*K+1)*theoric_theta)**2, Phi['Probability'].iloc[1])

))

So probabilities from $\hat{Q}^{K}|\Psi\rangle$ and from $\theta_{th}$ are the same but we cannot calculates the correct angle from the first because of multivalued $\arcsin$

## 3. Quantum Fourier Transformation

In order to solve the before section problem we can use the **Phase Estimation Algorithm**. 

Our operator $\hat{Q}$ applies a rotation of an angle $\theta$ on the state $|\Psi\rangle$. This operator is unitary and have 2 eigenvalues in form of phases: $\lambda_{\pm}=e^{\pm i2\theta}$. The **Phase Estimation Algorithm** allow us get this phase $\theta$.

For this we need to create n auxiliary qbits in state $|+\rangle$. Each qbit will be the controlled qbit for a controlled application of $\hat{Q}^{K}$ (for each controlled qbit the K will change). Finally over the auxiliary qbits we apply an inverse of the Quantum Fourier Operator ($\hat{Q}_{FT}$). The measurmente of this auxiliary qbits will give us an estimation of the desired $\theta$. This estimation will be more exact when more auxiliary qbits we use for the $\hat{Q}_{FT}$.

For this algorithm if we have $m$ auxiliary qbits and the measurment of the $\hat{Q}_{FT}$ is the integer $M$ then:

$$\theta = \frac{M\pi}{2^m}$$


In [ ]:
from qat.lang.AQASM import Program, H
from qat.lang.AQASM.qftarith import QFT

In [ ]:
#Allocating Auxiliar qbits for Phase Estimation
n_aux = 10

#Gate for loading data staff
PR_gate = expectation_loading_data(p_X, f_X)
#Groover Operator Gate
Q_Gate = load_q_gate(PR_gate)

#Loading Data in Quantum Circuit
Qprog = Program()
qbits = Qprog.qalloc(PR_gate.arity)
Qprog.apply(PR_gate, qbits)

#Creating auxiliary qbits for QPE with QFT
qAux = Qprog.qalloc(n_aux)
#QPE estimation procedure
for i, aux in enumerate(qAux):
    Qprog.apply(H, aux)
    #Gate for applying Q_gate 2^i times
    step_q_gate = load_qn_gate(Q_Gate, 2**i)
    #Controlled application mandatory for QPE
    Qprog.apply(step_q_gate.ctrl(), aux, qbits)    

#Inverse of the Quantum Fourier Transformation        
Qprog.apply(QFT(n_aux).dag(), qAux)
circuit = Qprog.to_circ(submatrices_only=True)

In [ ]:
%qatdisplay circuit --depth 0

In [ ]:
#Posible results
job = circuit.to_job(qubits=qAux)
result = run_job(lineal_qpu.submit(job))
Phi = postprocess_results(result)
#Each posible state is an integer M and we can compute its associated theta
Phi['Thetas'] = [m*np.pi/(2**n_aux) for m in list(Phi.index)] 
Phi.sort_values('Probability', ascending=False, inplace = True)

Now in Phi we have a complete pandas DataFrame with the posible results of the circuit:

In [ ]:
Phi.head()

We can plot the probabilities for each posible $\theta$ and we have to obtain 2 máximums of probability around the two atuovalues of $\hat{Q}$: $\pm \theta$

In [ ]:
#Now we can plot te probability for each posible theta
%matplotlib inline
plt.plot(Phi['Thetas'], Phi['Probability'], 'o')
plt.xlabel('Theta')
plt.ylabel('Probability')

As can be seen there are two maximum of probabilities at: $\theta_0$ and $\theta_1$ and is mandatory that $\theta_1 = -\theta_0$.

Remenbering that $-\theta = \pi - \theta$ the we can test that the two maximum correspond to $\pm \theta$:

In [ ]:
#We take the thetas were probability is maximum
theta_0 = Phi.sort_values('Probability', ascending =False)['Thetas'].iloc[0]
theta_1 = Phi.sort_values('Probability', ascending =False)['Thetas'].iloc[1]

print('theta_0: {}'.format(theta_0))
print('theta_1: {}'.format(theta_1))

In [ ]:
#So we test that the 2 are the same angle with different signs
np.isclose(theta_0, np.pi -theta_1)

Aditionally we can compute the Expected value as 
$$E_{x\sim p}(f) = \sum_{x=0}^{2^-1} p(x)f(x) = \sin^{2} \theta $$

We can plot the posible expected values versus the probability

In [ ]:
#Now we can plot te probability for each posible theta
%matplotlib inline
plt.plot(np.sin(Phi['Thetas'])**2, Phi['Probability'], 'o')
plt.xlabel('E_p[f]')
plt.ylabel('Probability')

In [ ]:
theoric_theta = np.arcsin(np.sqrt(sum(p_X*f_X)))
print('theoric_theta: {}'.format(theoric_theta))
print('theta_1: {}'.format(theta_0))
print('Theorical Integration: {}'.format(sum(p_X*f_X)))
print('Amplitude Amplification Integral: {}'.format(np.sin(theta_0)**2))